<a href="https://colab.research.google.com/github/Vignesh424/Weather-Prediction/blob/main/Weather_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [5]:
data = pl.read_csv('/content/weather_classification_data.csv')

In [6]:
data.head()

Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
f64,i64,f64,f64,str,f64,i64,str,f64,str,str
14.0,73,9.5,82.0,"""partly cloudy""",1010.82,2,"""Winter""",3.5,"""inland""","""Rainy"""
39.0,96,8.5,71.0,"""partly cloudy""",1011.43,7,"""Spring""",10.0,"""inland""","""Cloudy"""
30.0,64,7.0,16.0,"""clear""",1018.72,5,"""Spring""",5.5,"""mountain""","""Sunny"""
38.0,83,1.5,82.0,"""clear""",1026.25,7,"""Spring""",1.0,"""coastal""","""Sunny"""
27.0,74,17.0,66.0,"""overcast""",990.67,1,"""Winter""",2.5,"""mountain""","""Rainy"""


In [7]:
data.schema

Schema([('Temperature', Float64),
        ('Humidity', Int64),
        ('Wind Speed', Float64),
        ('Precipitation (%)', Float64),
        ('Cloud Cover', String),
        ('Atmospheric Pressure', Float64),
        ('UV Index', Int64),
        ('Season', String),
        ('Visibility (km)', Float64),
        ('Location', String),
        ('Weather Type', String)])

In [8]:
data.null_count()

Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0


In [9]:
le = LabelEncoder()
new_cleaned = data.with_columns(
    pl.Series(le.fit_transform(data['Cloud Cover'])).alias('Cloud Cover').cast(pl.Int64),
    pl.Series(le.fit_transform(data['Season'])).alias('Season').cast(pl.Int64),
    pl.Series(le.fit_transform(data['Location'])).alias('Location').cast(pl.Int64)
)

In [10]:
X = new_cleaned.drop('Weather Type')
Y = new_cleaned['Weather Type']

In [11]:
X_train , X_test , Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [12]:
dt = DecisionTreeClassifier()
dt.fit(X,Y)

DecisionTreeClassifier()

In [13]:
Y_Pred = dt.predict(X_test)

In [14]:
classification_report(Y_test, Y_Pred)

'              precision    recall  f1-score   support\n\n      Cloudy       1.00      1.00      1.00       797\n       Rainy       1.00      1.00      1.00       822\n       Snowy       1.00      1.00      1.00       865\n       Sunny       1.00      1.00      1.00       816\n\n    accuracy                           1.00      3300\n   macro avg       1.00      1.00      1.00      3300\nweighted avg       1.00      1.00      1.00      3300\n'